# Automated Multiple Reaction Monitoring (MRM)-profiling and Ozone Electrospray Ionizaton (OzESI)-MRM Informatics Platform for High-throughput Lipidomics


In this jupyter notebook you will automate the data analysis of the lipidome. This is a challenging problem to perform manually due to the diverse nature of lipids and the many potential isomers. In this notebook you will analyze mzML files containing data from lipid MRMs, with ozone off and ozone on. The goal is to identify possible double-bond locations in a lipid, in this case a TAG (triacylglycerols).

In [44]:
from IPython.display import Image

![title](Figures/agilent_lcms.png)

The examples shown here were run on an Agilent 6495C Triple Quadrupole LC/MS (example shown above) that has been connected to an ozone line (not shown in picture) for ozoneolysis of lipids.

![title](Figures/TAG_example.png)
Here is an example of a TAG. Notice how many possibilities there are for locations of one double-bond there could be and how convoluted the analysis can become! This image is obtained from LipidMaps.org

Import all necessary libraries

In [67]:
#Import all the necessary libraries
import pymzml
import csv
import os
import pandas as pd
import numpy as np
import math
from matplotlib import pyplot as plt
import re
import plotly.express as px

MAKE CLASSES FOR EACH LIPID

In [68]:
# lipid_types = ["CE","TAG","CER","FFA","PC","PE","PG","PI","SM","AC"]
# database_path = "lipid_database/Lipid_Database.xlsx"

# #loop through all sheets in SUPPLE_2.XLS and make a df of Compound Name, Parent Ion, and Product Ion
# mrm_list_new = pd.read_excel('lipid_database/Lipid_Database.xlsx', sheet_name = None)
# mrm_list_new = pd.concat(mrm_list_new, ignore_index=True)
# mrm_list_offical = mrm_list_new[['Compound Name', 'Parent Ion', 'Product Ion', 'Class']]
# #Add underscore to middle of columns names
# mrm_list_offical.columns = mrm_list_offical.columns.str.replace(' ', '_')
# #round Parent Ion and Product Ion to 1 decimal place
# mrm_list_offical['Parent_Ion'] = np.floor(mrm_list_offical['Parent_Ion'].round(1))
# mrm_list_offical['Product_Ion'] = np.floor(mrm_list_offical['Product_Ion'].round(1))
# #create transition column by combining Parent Ion and Product Ion with arrow between numbers
# mrm_list_offical['Transition'] = mrm_list_offical['Parent_Ion'].astype(str) + ' -> ' + mrm_list_offical['Product_Ion'].astype(str)
# #change column compound name to lipid
# mrm_list_offical = mrm_list_offical.rename(columns={'Compound_Name': 'Lipid'})
# #make a column called Class match lipid column to lipid types



# pd.set_option('display.max_rows', None)
# print(mrm_list_offical.head(None))


In [61]:

def read_mrm_list(filename):
    mrm_list_new = pd.read_excel(filename, sheet_name=None)
    mrm_list_new = pd.concat(mrm_list_new, ignore_index=True)
    mrm_list_offical = mrm_list_new[['Compound Name', 'Parent Ion', 'Product Ion', 'Class']]
    # Add underscore to middle of columns names
    mrm_list_offical.columns = mrm_list_offical.columns.str.replace(' ', '_')
    # Round Parent Ion and Product Ion to 1 decimal place
    mrm_list_offical['Parent_Ion'] = np.floor(mrm_list_offical['Parent_Ion'].round(1))
    mrm_list_offical['Product_Ion'] = np.floor(mrm_list_offical['Product_Ion'].round(1))
    # Create transition column by combining Parent Ion and Product Ion with arrow between numbers
    mrm_list_offical['Transition'] = mrm_list_offical['Parent_Ion'].astype(str) + ' -> ' + mrm_list_offical['Product_Ion'].astype(str)
    # Change column compound name to lipid
    mrm_list_offical = mrm_list_offical.rename(columns={'Compound_Name': 'Lipid'})
    # Make a column called Class match lipid column to lipid types
    return mrm_list_offical

mrm_database = read_mrm_list('lipid_database/Lipid_Database.xlsx')
mrm_database.tail()

/tmp/ipykernel_7529/2289239920.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mrm_list_offical['Parent_Ion'] = np.floor(mrm_list_offical['Parent_Ion'].round(1))
/tmp/ipykernel_7529/2289239920.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mrm_list_offical['Product_Ion'] = np.floor(mrm_list_offical['Product_Ion'].round(1))
/tmp/ipykernel_7529/2289239920.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value 

,Lipid,Parent_Ion,Product_Ion,Class,Transition
3264,STD_15:0-18:1(d7) PI (NH4 Salt),847.0,570.0,STD_15:0-18:1(d7) PI (NH4 Salt),847.0 -> 570.0
3265,STD_15:0-18:1(d7) PS (Na Salt),755.0,570.0,STD_15:0-18:1(d7) PS (Na Salt),755.0 -> 570.0
3266,STD_15:0-18:1(d7)-15:0 TAG,829.0,570.0,STD_15:0-18:1(d7)-15:0 TAG,829.0 -> 570.0
3267,STD_18:1(d7) Chol Ester,675.0,369.0,STD_18:1(d7) Chol Ester,675.0 -> 369.0
3268,STD_d18:1-18:1(d9) SM,738.0,184.0,STD_d18:1-18:1(d9) SM,738.0 -> 184.0


In [62]:
list_of_lipid_classes = mrm_database['Class'].unique()
print(list_of_lipid_classes)

['PC' 'PE' 'SM' 'Cer' 'CAR' 'TAG' 'DAG' 'PS' 'PI' 'PG' 'CE' 'FA'
 'STD 15:0-18:1-d7 DG' 'STD 18:1 (d7) Lyso PC' 'STD 18:1 (d7) Lyso PE'
 'STD 18:1(d7) MAG' 'STD C15 ceramide-D7' 'STD_15:0-18:1(d7) PC'
 'STD_15:0-18:1(d7) PE' 'STD_15:0-18:1(d7) PG (Na Salt)'
 'STD_15:0-18:1(d7) PI (NH4 Salt)' 'STD_15:0-18:1(d7) PS (Na Salt)'
 'STD_15:0-18:1(d7)-15:0 TAG' 'STD_18:1(d7) Chol Ester'
 'STD_d18:1-18:1(d9) SM']


Load mzML file and convert to pandas dataframe and csv file. |
Columns = Q1, Q3, Intensity, Transition, Lipid, Class  |
Parsed data is also stored as csv file in data_csv

In [74]:
#Create for loop to load all mzml files from the data folder into the run object from pymzml reader function and store in pandas dataframe
#Create empty dictionary to store all the data
df_OzESI = pd.DataFrame(columns=['Lipid','Parent_Ion','Product_Ion','Intensity','Retention_Time','Transition','Class','Sample_ID'])
###
# OzESI_time = {}
###

data_folder = os.listdir('./data_mzml/canola/test1/') #Path to the mzml files
path_to_mzml_files = './data_mzml/canola/test1/' #Path to the mzml files
#data_dict = {} #Empty dictionary to store all the data
df = pd.DataFrame(columns=['Lipid','Parent_Ion','Product_Ion','Intensity','Transition','Class','Sample_ID'])
#Create a similar for loop, except store all data in a single pandas dataframe
df_all = pd.DataFrame(columns=['Lipid','Parent_Ion','Product_Ion','Intensity','Transition','Class','Sample_ID']) #Create empty pandas dataframe to store the data
#df_all = pd.DataFrame(columns=['Q1','Q3','Intensity','Transition','Lipid','Class']) #Create empty pandas dataframe to store the data
for file in data_folder:
        if file.endswith('.mzML'):
                print(file)
                run = pymzml.run.Reader(path_to_mzml_files+file, skip_chromatogram=False) #Load the mzml file into the run object
                print('Spectrum # = ',run.get_spectrum_count())
                print('Chromatogram # =',run.get_chromatogram_count())


                
                #create pandas dataframe to store the data with the columns Parent Ion, Product Ion, Intensity, Transition Lipid and Class
                #df_sample = pd.DataFrame(columns=['Parent_Ion','Product_Ion','Intensity','Transition','Lipid','Class']) #Create empty pandas dataframe to store the data
                #df_sample = pd.DataFrame(columns=['Q1','Q3','Intensity','Transition','Lipid','Class']) #Create empty pandas dataframe to store the data
                q1_mz = 0 #Create empty variables to store the Q1 and Q3 m/z values
                q3_mz = 0
                count = 0 #Create a counter to keep track of the number of transitions
                for spectrum in run:
                        
                        ###
                        # if isinstance(spectrum,pymzml.spec.Chromatogram):
                        #         for time, intensity in spectrum.peaks():
                        #                 print(time, intensity)
                        #                 OzESI_time[time] = intensity
                        #         # OzESI_time.append(time_list)
                        ###

                        for element in spectrum.ID.split(' '):
                                # print('element',element)
                                intensity_store = np.array([])
                                if 'Q1' in element:
                                        #print('Q1',element)
                                        q1 = element.split('=')
                                        #print('q1',q1[1])
                                        q1_mz= np.floor(round(float(q1[1]),1))
                                        # print('q1',q1)
                                
                                if 'Q3' in element:
                                        #print('Q3',element)
                                        q3 = element.split('=')
                                        #print('q3',q3[1])
                                        q3_mz=np.floor(round(float(q3[1]),1))
                                        # print('q3',q3)
                                        # df_sample.loc[count,'Q1'] = q1_mz
                                        # df_sample.loc[count,'Q3'] = q3_mz
                                        
                                        for mz,intensity in spectrum.peaks(): #Get the m/z and intensity values from the spectrum
                                                intensity_store = np.append(intensity_store,intensity) #Store the intensity values in an array
                                                
                        
                                
                                if 'Q3' in element:
                                        # print(intensity_sum)
                                        intensity_sum = np.sum(intensity_store) #Sum the intensity values
                                        df_all.loc[count,'Parent_Ion'] = q1_mz #Store the Q1 and Q3 m/z values in the pandas dataframe
                                        df_all.loc[count,'Product_Ion'] = q3_mz
                                        #round the Q1 and Q3 m/z values to 1 decimal places
                                        df_all.loc[count,'Parent_Ion'] = np.floor(round(df_all.loc[count,'Parent_Ion'],1))
                                        df_all.loc[count,'Product_Ion'] = np.floor(round(df_all.loc[count,'Product_Ion'],1))
                                        df_all.loc[count,'Intensity'] = intensity_sum #Store the intensity values in the pandas dataframe
                                        df_all.loc[count,'Transition'] = str(q1_mz)+ ' -> '+ str(q3_mz) #Store the transition values in the pandas dataframe
                                        #add file name to Sample_ID column without the mzmL extension
                                        df_all.loc[count,'Sample_ID'] = file[:-5]
                                        count+=1
        #append df_all to df_all2
        df = df.append(df_all, ignore_index=True)

TailoredTAG18-1_O3off_RBDCanola0.0005mgmL_020723 (1).mzML
Spectrum # =  None
Chromatogram # = 36


/tmp/ipykernel_7529/1517786442.py:78: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(df_all, ignore_index=True)


In [71]:
# from scipy.signal import find_peaks
# import matplotlib.pyplot as plt

# print(len(OzESI_time))
# print(list(OzESI_time.items())[:3])
# test = pd.DataFrame(list(OzESI_time.items()), columns=['time', 'intensity'])
# print(test.head())
# #find peaks from test dataframe
# peaks, _ = find_peaks(test['intensity'], height=1e5,distance=1000)
# print(peaks)
# plt.plot(test['intensity'])
# plt.plot(peaks, test['intensity'][peaks], "x")
# plt.show()

In [75]:
df.tail(5) #Print the pandas dataframe

,Lipid,Parent_Ion,Product_Ion,Intensity,Transition,Class,Sample_ID
30,NaN,896.0,597.0,3145901.463318,896.0 -> 597.0,NaN,TailoredTAG18-1_O3off_RBDCanola0.0005mgmL_0207...
31,NaN,898.0,599.0,12535827.163513,898.0 -> 599.0,NaN,TailoredTAG18-1_O3off_RBDCanola0.0005mgmL_0207...
32,NaN,900.0,601.0,14229465.170547,900.0 -> 601.0,NaN,TailoredTAG18-1_O3off_RBDCanola0.0005mgmL_0207...
33,NaN,902.0,603.0,34867382.116547,902.0 -> 603.0,NaN,TailoredTAG18-1_O3off_RBDCanola0.0005mgmL_0207...
34,NaN,904.0,605.0,12858486.610764,904.0 -> 605.0,NaN,TailoredTAG18-1_O3off_RBDCanola0.0005mgmL_0207...


Load MRM transitions from csv file to pandas dataframe. This list will be used to identify the possible lipids in our sample.

In [ ]:
# #Match Ions in df to mrm_database and append Lipid and Class columns to df
# for index in range(len(df)):
#     for row in range(len(mrm_database)):
#         if mrm_database.loc[row,'Parent_Ion'] == df.loc[index,'Parent_Ion'] and mrm_database.loc[row,'Product_Ion'] == df.loc[index,'Product_Ion']:
#             df.loc[index,'Lipid'] = mrm_database.loc[row,'Lipid']
#             df.loc[index,'Class'] = mrm_database.loc[row,'Class']

# df_matching = df.dropna() #drop rows with NaN values
            

In [76]:
# create a dictionary to store the matching parent/product ion pairs and their corresponding lipids and classes
ion_dict = {}
for index, row in mrm_database.iterrows():
    ion_dict[(row['Parent_Ion'], row['Product_Ion'])] = (row['Lipid'], row['Class'])

# create empty columns for Lipid and Class
df['Lipid'] = np.nan
df['Class'] = np.nan

# loop through the rows in df and check if the corresponding parent/product ion pair exists in ion_dict
for index, row in df.iterrows():
    if (row['Parent_Ion'], row['Product_Ion']) in ion_dict:
        df.at[index, 'Lipid'] = ion_dict[(row['Parent_Ion'], row['Product_Ion'])][0]
        df.at[index, 'Class'] = ion_dict[(row['Parent_Ion'], row['Product_Ion'])][1]

# drop rows with NaN values
df_matching = df.dropna()


In [77]:
df_matching.head(None)

,Lipid,Parent_Ion,Product_Ion,Intensity,Transition,Class,Sample_ID
0,PG(34:4),760.0,571.0,3508930.004333,760.0 -> 571.0,PG,TailoredTAG18-1_O3off_RBDCanola0.0005mgmL_0207...
1,"PG(34:3),PG(P-35:2)",762.0,573.0,4454522.478516,762.0 -> 573.0,PG,TailoredTAG18-1_O3off_RBDCanola0.0005mgmL_0207...
2,"PG(O-35:2),PG(P-35:1)",764.0,575.0,3342623.235199,764.0 -> 575.0,PG,TailoredTAG18-1_O3off_RBDCanola0.0005mgmL_0207...
3,"PG(O-35:1),PG(P-35:0)",766.0,577.0,3270586.69046,766.0 -> 577.0,PG,TailoredTAG18-1_O3off_RBDCanola0.0005mgmL_0207...
4,"PG(36:7),PG(35:0),PG(O-36:0)",782.0,593.0,2358947.425293,782.0 -> 593.0,PG,TailoredTAG18-1_O3off_RBDCanola0.0005mgmL_0207...
5,PG(36:6),784.0,595.0,2304902.763245,784.0 -> 595.0,PG,TailoredTAG18-1_O3off_RBDCanola0.0005mgmL_0207...
7,PG(36:5),786.0,597.0,3260273.493011,786.0 -> 597.0,PG,TailoredTAG18-1_O3off_RBDCanola0.0005mgmL_0207...
9,PG(36:4),788.0,599.0,6129489.851624,788.0 -> 599.0,PG,TailoredTAG18-1_O3off_RBDCanola0.0005mgmL_0207...
10,"PG(36:3),PG(P-37:2)",790.0,601.0,2584182.920563,790.0 -> 601.0,PG,TailoredTAG18-1_O3off_RBDCanola0.0005mgmL_0207...
12,"PG(O-37:2),PG(P-37:1)",792.0,603.0,3311812.273865,792.0 -> 603.0,PG,TailoredTAG18-1_O3off_RBDCanola0.0005mgmL_0207...


In [ ]:
#for loop to check if that file already exists and if it does then add a number to the end of the file name so data is not overwritten
name_of_folder = 'canola'
name_of_file = 'canola'


for i in range(5):
    if not os.path.isfile('data_results/data/data_matching/'+name_of_folder+'/'+name_of_file+'_{}.xlsx'.format(i)):
        df_matching.to_excel('data_results/data/data_matching/'+name_of_folder+'/'+name_of_file+'_{}.xlsx'.format(i), index=False)
        break
    elif not os.path.isfile('data_results/data/data_matching/'+name_of_folder+'/'+name_of_file+'_{}.xlsx'.format(i)):
        df_matching.to_excel('data_results/data/data_matching/'+name_of_folder+'/'+name_of_file+'_{}.xlsx'.format(i), index=False)
        break
    else:
        pass
df_matching.tail(5)

In [79]:
#import visualization libraries
import umap
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go

/home/sanjay/anaconda3/envs/pymzml/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning:

IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html



In [81]:
#plot transition versus intensity of df_matching
fig = px.bar(df_matching, x="Transition", y="Intensity", color="Lipid", hover_data=['Lipid','Class'])
fig.show()


#plot lipid class versus intensity of df_matching in a bar chart
fig = px.bar(df_matching, x="Class", y="Intensity", color="Class", hover_data=['Lipid','Class'])
fig.show()
#plot lipid class versus intensity of df_matching in a pie chart
fig = px.pie(df_matching, values='Intensity', names='Class', title='Lipid Class')
fig.show()
#make a plotly heatmap of the intensity of each transition in each sample
fig = go.Figure(data=go.Heatmap(
                     z=df_matching['Intensity'],
                        x=df_matching['Lipid'],
                        y=df_matching['Class'],
                        colorscale='Viridis'))
fig.show()

# #plot sample ID versus intensity of df_matching
# fig = px.bar(df_matching, x="Sample_ID", y="Intensity", color="Sample_ID", hover_data=['Lipid','Class'])
# fig.show()
# #plot sample ID versus intensity of df_matching in a pie chart
# fig = px.pie(df_matching, values='Intensity', names='Sample_ID', title='Sample ID')
# fig.show()